In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.util import bigrams
import geopandas as gpd

In [11]:
df_complete = pd.read_excel('results/final/complete.xlsx', sheet_name='AI_ICSE2025_papers')

df_relevant = df_complete[df_complete['relevant'] == True]

print("Columns", df_complete.columns.tolist())

Columns ['reviewer', 'relevant', 'year', 'title', 'authors', 'url', 'abstract', 'artifact_available', 'artifact_reusable', 'artifact_functional', 'ai', 'Task being solved  (CODE GENERATION;APR;TESTING;COMMENT GENERATION/...)', 'AA Task Consolidated', 'Do the papers evaluate with approaches other than LLM? (TRUE/FALSE)', 'Is the budget fair (on hold)', 'Do they evaluate with open source/commercial/open weight models? (COMMERCIAL/OPEN SOURCE/OPEN WEIGHT)', 'DW Open/Closed Consolidated ', 'How many models overall, benchmarks and approach? (integer)', 'DW Num. Models Conslidated', 'Name of models', 'DW Scale of models (LARGE >50B/MEDIUM>1B/SMALL<1B)', 'Size of models from the paper (free text from the paper)', 'DW Model Sizes Reported (none, partial, full, N/A for commercial)', 'Do they report on the model versions? (TRUE/FALSE)', 'Configuration of models (TRUE/FALSE, or parameters)', 'AA Configuration consolidation', 'Type of dataset (code, text, logs, documentation, images)', 'Programmin

In [12]:
df_complete['relevant'].value_counts()

relevant
True       164
False       50
SPECIAL      3
Name: count, dtype: int64

In [13]:
# Number of relevant papers per year
df_relevant['year'].value_counts()

year
2025.0    89
2024.0    53
2023.0    22
Name: count, dtype: int64

In [14]:
print(df_relevant["AA Task Consolidated"].unique())

['vulnerability detection' 'test generation' 'test repair'
 'code refinement' 'code generation' 'comment repair' 'program repair'
 'SO post editing' 'code reasoning; code translation' 'code reasoning'
 'code translation' 'bug detection' 'security patch detection'
 'model completion' 'commit message generation'
 'traceability link recovery' 'log parsing' 'AI generated code detection'
 'code generation; code summarization' 'program analysis'
 'regression testing' 'code search' 'fuzzing'
 'inconsistency prediction in decentralised apps' 'formal verification'
 'bug report comprehension' 'UI design repair' 'type detection'
 'smart contract auditing' 'configuration validation'
 'detection of code design issues' 'code understanding; code generation'
 'code completion' 'clone detection; vulnerability detection'
 'code adaptation' 'fault localisation; bug detection'
 'code summarization' 'code optimisation' 'security injection'
 'machine-generated code detection' 'root cause analysis'
 'code su

In [15]:
# Split the 'AA Task Consolidated' column into lists
df_relevant['AA Task List'] = df_relevant['AA Task Consolidated'].apply(
    lambda x: [task.strip() for task in str(x).split(';')] if pd.notna(x) else []
)

df_relevant[['AA Task Consolidated', 'AA Task List']].head(50)

/var/folders/gw/jg8766954y92j4x7kpzdhg680000gp/T/ipykernel_34007/1597075678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_relevant['AA Task List'] = df_relevant['AA Task Consolidated'].apply(


,AA Task Consolidated,AA Task List
0,vulnerability detection,[vulnerability detection]
1,test generation,[test generation]
2,test repair,[test repair]
4,code refinement,[code refinement]
6,test generation,[test generation]
7,code generation,[code generation]
9,test generation,[test generation]
10,code generation,[code generation]
11,test generation,[test generation]
13,code generation,[code generation]


In [16]:
from collections import Counter

all_tasks = []
for task_list in df_relevant['AA Task List']:
    all_tasks.extend(task_list)

task_counts = Counter(all_tasks)

task_frequency_df = pd.DataFrame(task_counts.items(), columns=['Task', 'Paper_Count'])
task_frequency_df = task_frequency_df.sort_values('Paper_Count', ascending=False)

print(f"Total unique tasks: {len(task_frequency_df)}")
task_frequency_df

Total unique tasks: 68


,Task,Paper_Count
4,code generation,25
1,test generation,24
6,program repair,22
0,vulnerability detection,13
10,bug detection,8
...,...,...
7,SO post editing,1
5,comment repair,1
41,"clone detection, bug detection",1
42,API misuse detection,1
